*Trabajo Final, Alejandro Rodríguez Mesa. Autenticación por Reconomiento Facial*

# Entrenamientos de Modelos YOLO

### Entrenamiento para detección de gafas y máscaras o mascarillas

In [ ]:
from ultralytics import YOLO
model = YOLO('yolo11n.pt') #Contenedores
data_Yaml_path = "C:\\Users\\aleja\\OneDrive\\Documentos\\VC_Practicas_Alejandro_Rodriguez_Mesa\\glasses_and_mask\\data.yaml"

results = model.train(data= data_Yaml_path, epochs=40, imgsz=640, batch=16, lr0=0.001, patience=5)

### Prueba del modelo

In [1]:
import cv2
from ultralytics import YOLO
import mediapipe as mp
import logging

# Configurar logging para reducir los outputs
logging.getLogger("ultralytics").setLevel(logging.WARNING)

# Cargar el modelo YOLO entrenado
yolo_model = YOLO(r"runs\detect\train6\weights\best.pt")

# Mapeo de nombres de clases a sus índices
class_names = yolo_model.names

# Crear un conjunto de clases que queremos mostrar
classes_to_show = {'glasses', 'mask'}
# Obtener los índices de las clases que queremos mostrar
classes_indices = [index for index, name in class_names.items() if name in classes_to_show]

# Inicializar MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Inicializar la captura de video de la cámara
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()


while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el frame de la cámara.")
        break

    # Convertir la imagen a RGB para MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detectar caras con MediaPipe
    results = face_detection.process(rgb_frame)

    if results.detections:
        for detection in results.detections:
            # Extraer el bounding box de la cara
            bboxC = detection.location_data.relative_bounding_box
            ih, iw, _ = frame.shape
            x1, y1, w, h = (int(bboxC.xmin * iw), int(bboxC.ymin * ih),
                            int(bboxC.width * iw), int(bboxC.height * ih))
            x2, y2 = x1 + w, y1 + h

            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para la detección de caras
            # Ajustar las coordenadas para mantenerlas dentro de los límites del frame
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(frame.shape[1], x2)
            y2 = min(frame.shape[0], y2)

            # Recortar la región de la cara
            if x2 > x1 and y2 > y1:  # Asegurarse de que las dimensiones sean válidas
                face_roi = frame[y1:y2, x1:x2]

                if face_roi.size > 0:  # Verificar que el ROI no esté vacío
                    # Procesar la región de la cara con YOLO
                    yolo_results = yolo_model.predict(face_roi, imgsz=640, conf=0.3)

                    # Dibujar detecciones de YOLO en la región de la cara
                    for result in yolo_results:
                        boxes = result.boxes
                        for box in boxes:
                            cls_id = int(box.cls)
                            if cls_id in classes_indices:
                                fx1, fy1, fx2, fy2 = map(int, box.xyxy[0])
                                confidence = box.conf[0]
                                class_name = class_names[cls_id]

                                # Ajustar las coordenadas a la posición en el frame original
                                fx1 += x1
                                fy1 += y1
                                fx2 += x1
                                fy2 += y1

                                # Dibujar la bounding box y el texto
                                color = (0, 255, 0)  # Verde
                                cv2.rectangle(frame, (fx1, fy1), (fx2, fy2), color, 2)
                                label = f"{class_name} {confidence:.2f}"
                                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                                cv2.rectangle(frame, (fx1, fy1 - 20), (fx1 + w, fy1), color, -1)
                                cv2.putText(frame, label, (fx1, fy1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Mostrar el frame con las detecciones
    cv2.imshow('Detecciones YOLO', frame)

    # Salir del loop si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


### Entrenamiento modelo de detección de Ataques de Presentacion

Se detectan imágenes, máscaras y dispositivos que pueden ser usados en spoofing. 

In [ ]:
#from ultralytics import YOLO
#model2 = YOLO('yolo11n.pt') #Contenedores
#model2_Yaml_path = "C:\\Users\\aleja\\OneDrive\\Documentos\\VC_Practicas_Alejandro_Rodriguez_Mesa\\spoofing\\data.yaml"

#results_model2 = model2.train(data= model2_Yaml_path, epochs=40, imgsz=640, batch=16, lr0=0.001, patience=5)

Prueba del Entrenamiento

In [ ]:
import cv2
from ultralytics import YOLO

# Cargar el modelo YOLO entrenado
yolo_model2 = YOLO(r"runs\detect\train5\weights\best.pt")
# Mapeo de nombres de clases a sus índices
# Asegúrate de que estos índices coincidan con los de tu modelo
# Si no estás seguro, puedes imprimir model.names para verificar

class_names = yolo_model2.names
print(class_names)
# Crear un conjunto de clases que queremos mostrar
classes_to_show = {'Device', 'Masks', 'Photo'}
# Obtener los índices de las clases que queremos mostrar
classes_indices = [index for index, name in class_names.items() if name in classes_to_show]

# Inicializar la captura de video de la cámara (0 es generalmente la cámara predeterminada)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Presiona 'q' para salir.")


while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el frame de la cámara.")
        break

    # Ejecutar la detección con el modelo
    results = yolo_model2.predict(frame, imgsz=640, conf=0.25)

    # Procesar los resultados
    for result in results:
        boxes = result.boxes  # Bounding boxes
        for box in boxes:
            cls_id = int(box.cls)  # ID de la clase
            if cls_id in classes_indices:
                # Obtener coordenadas de la bounding box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                class_name = class_names[cls_id]

                # Dibujar la bounding box
                color = (0, 255, 0)  # Verde para las cajas
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Preparar el texto con el nombre de la clase y la confianza
                label = f"Spoofing - {class_name} {confidence:.2f}"

                # Obtener el tamaño del texto para un mejor posicionamiento
                (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                cv2.rectangle(frame, (x1, y1 - 20), (x1 + w, y1), color, -1)  # Fondo del texto
                cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Mostrar el frame con las detecciones
    cv2.imshow('Detecciones YOLO', frame)

    # Salir del loop si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


# Pruebas con detector facial Dlib

Al inicio tenía claro usar medaipipe, debido a la facilidad de uso, y la extensa malla de puntos que ofrece. Pero al ver que no habian buenos datasets anotados acerca de oclusión en caras, valoré el uso de detectores que sean más sensibles a oclusión. Dlib por ejemplo, tiende a no detectar caras cuando puntos clave como ojos o nariz están ocultos, mientras MediaPipe es menos sensible. Finalmente decidí usar MediaPipe, porque no compensaba la pequeña mejora en descartes ante oclusión a la vez que se perjudica el conteo de pestañeos, la precisión en la estimación de la posición de la cabeza, etcétera.

In [15]:
import cv2
import dlib
from ultralytics import YOLO
import logging
import numpy as np

# Configurar logging para reducir los outputs
logging.getLogger("ultralytics").setLevel(logging.WARNING)

# Cargar el modelo YOLO entrenado
yolo_model = YOLO(r"runs\detect\train6\weights\best.pt")

# Mapeo de nombres de clases a sus índices
class_names = yolo_model.names

# Crear un conjunto de clases que queremos mostrar
classes_to_show = {'glasses'}
# Obtener los índices de las clases que queremos mostrar
classes_indices = [index for index, name in class_names.items() if name in classes_to_show]

# Inicializar Dlib para detección facial y predicción de forma facial
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Función para calcular el aspecto de relación ocular (EAR)
def calculate_ear(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Función para dibujar puntos de landmarks
def draw_landmarks(canvas, landmarks):
    for (x, y) in landmarks:
        cv2.circle(canvas, (x, y), 2, (0, 255, 0), -1)

# Índices de los puntos de los ojos según el modelo de Dlib
LEFT_EYE = list(range(36, 42))
RIGHT_EYE = list(range(42, 48))

# Parámetros para detectar pestañeos
EAR_THRESHOLD = 0.25
CONSEC_FRAMES = 3
blink_count = 0
frame_counter = 0
prev_eye_state = "open"
eye_transition_flag = False

def extract_eye_coordinates(shape, eye_indices):
    return np.array([(shape.part(i).x, shape.part(i).y) for i in eye_indices], dtype="int")

def extract_landmarks(shape):
    return np.array([(shape.part(i).x, shape.part(i).y) for i in range(68)], dtype="int")

# Inicializar la captura de video de la cámara
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: No se pudo leer el frame de la cámara.")
        break

    # Convertir la imagen a escala de grises para Dlib
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detectar caras con Dlib
    faces = face_detector(gray_frame)

    for face in faces:
        # Obtener las coordenadas del bounding box de la cara
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()

        # Ajustar las coordenadas para mantenerlas dentro de los límites del frame
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(frame.shape[1], x2)
        y2 = min(frame.shape[0], y2)

        # Dibujar la bounding box de la cara en el frame original
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para la detección de caras

        # Predecir los puntos clave faciales
        shape = shape_predictor(gray_frame, face)

        # Extraer las coordenadas de los ojos
        left_eye = extract_eye_coordinates(shape, LEFT_EYE)
        right_eye = extract_eye_coordinates(shape, RIGHT_EYE)
        landmarks = extract_landmarks(shape)

        # Dibujar landmarks en la cara
        draw_landmarks(frame, landmarks)

        # Calcular el EAR para ambos ojos
        left_ear = calculate_ear(left_eye)
        right_ear = calculate_ear(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # Dibujar los ojos en el frame
        cv2.polylines(frame, [left_eye], True, (0, 255, 255), 1)
        cv2.polylines(frame, [right_eye], True, (0, 255, 255), 1)

        # Determinar el estado del ojo y contar pestañeos
        eyes_open_now = ear >= EAR_THRESHOLD

        if eyes_open_now:
            if prev_eye_state == "closed" and not eye_transition_flag:
                blink_count += 1
                eye_transition_flag = True
            prev_eye_state = "open"
        else:
            if prev_eye_state == "open":
                eye_transition_flag = False
            prev_eye_state = "closed"

    # Mostrar el conteo de pestañeos
    cv2.putText(frame, f"Pestaneos: {blink_count}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Mostrar el frame con las detecciones
    cv2.imshow('Detecciones YOLO', frame)

    # Salir del loop si se presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
cv2.destroyAllWindows()


# Importante - Comprobación de funcionamiento del análisis de imagenes de Deepface
Es necesario ejecutar previamente este código para asegurar que funciona el analyze. Puede ser necesario que se descarguen ciertas dependencias de la función. Si falla, se recomienda reiniciar el enviroment y volver a ejecutarlo. Una vez hecho eso, debería de funcionar. Si al clickar en el botón de "predicciones" en el login da error, se debe a que las dependencias no se han cargado correctamente, con lo que se debe ejecutar esto, y una vez que funcione, ya se podrá utilizar la funcionalidad.

Deepface


In [1]:
from deepface import DeepFace

# Analyze the image
demography = DeepFace.analyze("./usuarios/ale/foto_1.png")

# Access the first face's result
if isinstance(demography, list):  # If multiple faces are detected
    demography = demography[0]

print("Age: ", demography["age"])
print("Gender: ", demography["gender"])
print("Emotion: ", demography["dominant_emotion"])
print("Race: ", demography["dominant_race"])


Action: race: 100%|██████████| 4/4 [00:05<00:00,  1.41s/it]   

Age:  21
Gender:  {'Woman': 0.06050013471394777, 'Man': 99.93950128555298}
Emotion:  neutral
Race:  latino hispanic


# Estimación de la posición de la cabeza con MediaPipe

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializamos MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("No se pudo obtener la imagen de la cámara.")
        break

    start = time.time()

    # Convertimos la imagen a RGB y la volteamos horizontalmente
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face_mesh.process(image)

    # Volvemos a BGR para dibujar y mostrar
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, _ = image.shape

    # Listas para almacenar coordenadas 2D y 3D de puntos clave
    face_3d = []
    face_2d = []

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Recorremos cada landmark y seleccionamos los índices de interés
            for idx, lm in enumerate(face_landmarks.landmark):
                # Usamos varios puntos clave de la cara
                if idx in [33, 263, 1, 61, 291, 199]:
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    
                    # Guardamos coordenadas 2D
                    face_2d.append([x, y])
                    # Guardamos coordenadas 3D (notar que lm.z es relativo)
                    face_3d.append([x, y, lm.z])

                    # Para proyectar la dirección de la nariz
                    if idx == 1:  
                        nose_2d = (x, y)
                        nose_3d = (x, y, lm.z * 3000)

            # Convertimos las listas a arrays de NumPy
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)

            # Matriz de cámara (focal_length ~ ancho de la imagen)
            focal_length = 1 * img_w
            cam_matrix = np.array([
                [focal_length, 0, img_h / 2],
                [0, focal_length, img_w / 2],
                [0, 0, 1]
            ])

            # Sin distorsión
            dist_matrix = np.zeros((4, 1), dtype=np.float64)

            # Resolver PnP
            success_pnp, rot_vec, trans_vec = cv2.solvePnP(
                face_3d, face_2d, cam_matrix, dist_matrix
            )

            # Convertir a matriz de rotación
            rmat, _ = cv2.Rodrigues(rot_vec)

            # Obtener ángulos de rotación
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)

            x_angle = angles[0] * 360
            y_angle = angles[1] * 360
            z_angle = angles[2] * 360

            # Definir texto según la inclinación de la cabeza
            if y_angle < -4:
                text = "Mirando Izq."
            elif y_angle > 5:
                text = "Mirando Der."
            elif x_angle < -3:
                text = "Mirando Abajo"
            elif x_angle > 8:
                text = "Mirando Arriba"
            else:
                text = "Frente"

            # Proyectar la nariz para visualizar la dirección
            nose_3d_projection, _ = cv2.projectPoints(
                np.array([nose_3d]), rot_vec, trans_vec, cam_matrix, dist_matrix
            )

            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (
                int(nose_2d[0] + y_angle * 10),
                int(nose_2d[1] - x_angle * 10)
            )
            cv2.line(image, p1, p2, (255, 0, 0), 3)

            # Mostrar texto
            cv2.putText(image, text, (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, f'x: {round(x_angle, 2)}', (20, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, f'y: {round(y_angle, 2)}', (20, 130),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, f'z: {round(z_angle, 2)}', (20, 160),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


# Código Final

# Código Completo - Final

In [ ]:

#------------------------Sección 1: Importación de Paquetes---------------------------------

"""
En esta sección se importan todas las librerías necesarias para 
la aplicación. Entre ellas, librerías de interfaz gráfica (`tkinter`), 
visión artificial (`cv2`, `mediapipe`), librerías de utilidades como `numpy` 
y `json`, además de las librerías para el reconocimiento facial (`deepface`) 
y el modelado con YOLO (`ultralytics`). 
Por último, se maneja la configuración de logs para suprimir mensajes innecesarios.
"""

import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk, ImageSequence, ImageDraw, ImageFont
import cv2
import mediapipe as mp
import threading
import os
import numpy as np
import time
import json
import shutil
import math

from deepface import DeepFace
from deepface.modules import verification as df_verification

import logging
logging.getLogger("ultralytics").setLevel(logging.CRITICAL)

import pygame
from ultralytics import YOLO

#-------------------Sección 2: Funciones Auxiliares (Helper Functions)--------------------------------
"""
En esta sección se definen funciones sueltas que ayudan a procesar 
imágenes, dibujar texto, calcular distancias o manejar la reproducción de audio. 
Esto sirve para mantener el código más organizado.
Copiar código
"""
# Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

def enhance_image_cv2(bgr_img):
    """Mejora la imagen BGR aplicando ecualización del canal Y (YUV)."""
    yuv_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2YUV)
    yuv_img[:, :, 0] = cv2.equalizeHist(yuv_img[:, :, 0])
    enhanced = cv2.cvtColor(yuv_img, cv2.COLOR_YUV2BGR)
    return enhanced

def put_text_pil(
    image_bgr,
    text,
    org=(30, 30),
    font_path="arial.ttf",
    font_size=28,
    color=(255, 255, 255)
):
    """
    Dibuja texto (incluyendo caracteres latinos) sobre una imagen BGR usando PIL.
    Asegúrate de tener un archivo de fuente TrueType que soporte tildes/ñ.
    """
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(image_rgb)
    draw = ImageDraw.Draw(pil_img)

    try:
        font = ImageFont.truetype(font_path, font_size)
    except:
        font = ImageFont.load_default()

    draw.text(org, text, font=font, fill=color)
    image_bgr = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    return image_bgr

def euclidean_distance(vec1, vec2):
    """Calcula la distancia Euclidiana entre dos vectores 1D de igual tamaño."""
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    return np.sqrt(np.sum((v1 - v2) ** 2))

def manhattan_distance(vec1, vec2):
    """Calcula la distancia Manhattan entre dos vectores 1D de igual tamaño."""
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    return np.sum(np.abs(v1 - v2))

# Sección 3: Clase Principal de la Aplicación
"""
Aquí se encuentra la clase `App` que encapsula toda la lógica de la interfaz gráfica, 
el control de la cámara, la detección y reconocimiento facial, la reproducción de audio, 
el manejo de registros (log) y la navegación entre pantallas de Sign Up, Login, Perfil, 
Estadísticas, etc.
"""

class App(tk.Tk):
    """
    Clase principal de la aplicación de reconocimiento facial.
    """
    def __init__(self):
        super().__init__()
        self.resizable(False, False)
        self.title("Sistema de Reconocimiento Facial")
        self.geometry("800x600")

        pygame.mixer.init()
        pygame.mixer.set_num_channels(8)

        self.lift()
        self.attributes("-topmost", True)
        self.after(100, lambda: self.attributes("-topmost", False))

        # *** CANAL para música de fondo
        self.channel_background = pygame.mixer.Channel(3)
        self.background_music = None

        # Modelos YOLO
        self.yolo_model = YOLO(r"runs\detect\train6\weights\best.pt")  # gafas/máscara
        self.yolo_model2 = YOLO(r"runs\detect\train5\weights\best.pt") # spoofing

        self.channel_instructions = pygame.mixer.Channel(0)
        self.channel_errors = pygame.mixer.Channel(1)

        # Audios
        self.camera_sound = pygame.mixer.Sound('./assets/sound/camera.mp3')
        self.audio_creador = pygame.mixer.Sound('./assets/sound/creador.mp3')
        self.audio_login = pygame.mixer.Sound('./assets/sound/login.mp3')
        self.audio_ojos = pygame.mixer.Sound('./assets/sound/ojos.mp3')
        self.audio_boca = pygame.mixer.Sound('./assets/sound/boca.mp3')

        self.ultimo_sonido_ojos = 0
        self.ultimo_sonido_boca = 0
        self.audio_cooldown = 15

        self.camera_ready = False
        self.cap = None

        # FaceMesh y FaceDetection
        self.face_mesh = None
        self.face_detection = None

        # Atributo para saber si estamos en medio de un SignUp
        self.in_signup_process = False

        # SignUp
        self.username = ""
        self.ellipse_offsets = [(0, 0), (0, 100), (0, 200), (-150, 100), (150, 100)]
        self.current_ellipse_index = 0
        self.total_ellipses = len(self.ellipse_offsets)
        self.max_images = 5
        self.captured_count = 0
        self.required_stable_seconds = 2.0
        self.stable_start_time = None

        # *** Se define ellipse en base al frame
        self.fixed_rx = None
        self.fixed_ry = None
        self.ellipse_angle = 0
        self.ellipse_defined_for_this_offset = False

        # Login
        self.is_logging_in = False
        self.login_timeout_start = None
        self.blink_count = 0
        self.prev_eye_state = "open"
        self.eye_transition_flag = False
        self.login_image_captured = None
        self.login_embedding_vector = None
        self.login_start_time = None

        # Usuario logueado
        self.current_logged_in_user = None
        self.login_image_pil = None
        self.login_image_pil_unprocessed = None

        # Simetría y predicciones
        self.simmetry_label_var = tk.StringVar(value="")
        self.predictions_label_var = tk.StringVar(value="")

        # ---------------------------------------------------------------------
        #  MARCOS / FRAMES PRINCIPALES
        # ---------------------------------------------------------------------
        self.brand_video_frame = tk.Frame(self, bg="black")
        self.brand_video_frame.place(x=0, y=0, width=800, height=600)

        self.video_label_intro = tk.Label(self.brand_video_frame, bg="black")
        self.video_label_intro.pack(fill="both", expand=True)

        skip_button = tk.Button(
            self.brand_video_frame,
            text="Saltar Intro",
            command=self.skip_intro,
            bg="gray",
            fg="white"
        )
        skip_button.place(x=700, y=550, width=80, height=30)

        self.start_frame = tk.Frame(self, bg="#1e1e1e")
        self.signup_frame = tk.Frame(self, bg="#1e1e1e")
        self.login_frame = tk.Frame(self, bg="#1e1e1e")
        self.profile_frame = tk.Frame(self, bg="#2c2c2c")
        self.stats_frame = tk.Frame(self, bg="#3e3e3e")

        # Label principal para cámara
        self.video_label = tk.Label(self, bg="#1e1e1e")

        # Loader unificado
        self.loader_label = tk.Label(self, bg="#1e1e1e")
        self.loader_frames = []
        self.current_loader_frame = 0
        loader_path = "./assets/images/loader.gif"
        if os.path.exists(loader_path):
            loader_img = Image.open(loader_path)
            for frame in ImageSequence.Iterator(loader_img):
                f = frame.convert("RGBA")
                self.loader_frames.append(ImageTk.PhotoImage(f))

        self.flash_label = tk.Label(self, bg="white")


        self.last_spoof_box = None
        self.spoof_lost_count = 0
        self.spoof_lost_threshold = 5

        # Hilo para cámara
        threading.Thread(target=self.initialize_camera).start()

        # Manejo de cierre de ventana
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

        # Intro video
        self.intro_cap = None
        self.intro_audio = None
        self.is_intro_playing = True

        video_path = "./assets/images/video.mp4"
        audio_path = "./assets/images/musica-inicial.mp3"
        self.intro_cap = cv2.VideoCapture(video_path)

        if os.path.exists(audio_path):
            self.intro_audio = pygame.mixer.Sound(audio_path)
            self.intro_channel = pygame.mixer.Channel(2)
            self.intro_channel.play(self.intro_audio)

        self.update_intro_video()

    # --------------------------------------------------------------------------
    # Música de fondo
    # --------------------------------------------------------------------------
    def play_background_music(self, music_path="./assets/sound/light_background_music.mp3", volume=0.2):
        """Inicia la música de fondo en bucle con cierto volumen."""
        if not os.path.exists(music_path):
            return
        if self.background_music is None:
            self.background_music = pygame.mixer.Sound(music_path)
        self.background_music.set_volume(volume)
        self.channel_background.play(self.background_music, loops=-1)

    def stop_background_music(self):
        """Detiene la música de fondo."""
        if self.channel_background.get_busy():
            self.channel_background.stop()

    def pause_background_music(self):
        """Pausa la música de fondo (si está reproduciendo)."""
        if self.channel_background.get_busy():
            self.channel_background.pause()

    def resume_background_music(self):
        """Resume la música de fondo (si estaba en pausa)."""
        if not self.channel_background.get_busy():
            self.channel_background.play(self.background_music, loops=-1)
        else:
            self.channel_background.unpause()

    # --------------------------------------------------------------------------
    # Eventos de cierre
    # --------------------------------------------------------------------------
    def on_closing(self):
        # Detén todos los canales que has utilizado:
        self.channel_instructions.stop()
        self.channel_errors.stop()
        if hasattr(self, 'intro_channel'):
            self.intro_channel.stop()
        
        # Además de tu música de fondo
        self.stop_background_music()
        
        #Se limpian los labels
        self.simmetry_label_var = tk.StringVar(value="")
        self.predictions_label_var = tk.StringVar(value="")

        # Resto de tareas de limpieza
        if self.in_signup_process:
            self.cancel_signup()
        if self.cap and self.cap.isOpened():
            self.cap.release()
        self.destroy()

    # --------------------------------------------------------------------------
    # Loader (GIF)
    # --------------------------------------------------------------------------
    def animate_loader(self):
        if not self.loader_frames:
            return
        frame = self.loader_frames[self.current_loader_frame]
        self.loader_label.config(image=frame)
        self.current_loader_frame = (self.current_loader_frame + 1) % len(self.loader_frames)
        self.after(100, self.animate_loader)

    # --------------------------------------------------------------------------
    # Intro Video
    # --------------------------------------------------------------------------
    def update_intro_video(self):
        if not self.intro_cap:
            self.finish_intro()
            return

        ret, frame = self.intro_cap.read()
        if not ret:
            self.finish_intro()
            return

        frame = cv2.resize(frame, (800, 600))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        imgtk = ImageTk.PhotoImage(Image.fromarray(frame_rgb))
        self.video_label_intro.imgtk = imgtk
        self.video_label_intro.configure(image=imgtk)

        self.video_label_intro.after(33, self.update_intro_video)

    def finish_intro(self):
        """Se llama cuando termina el video de introducción."""
        self.is_intro_playing = False
        if self.intro_audio:
            self.intro_channel.stop()
        if self.intro_cap:
            self.intro_cap.release()
        self.intro_cap = None

        self.brand_video_frame.place_forget()
        self.create_start_frame_gui()
        self.start_frame.place(x=0, y=0, width=800, height=600)

        self.play_background_music(volume=0.2)

    def skip_intro(self):
        """Si el usuario presiona 'Saltar Intro' antes de terminar el video."""
        self.is_intro_playing = False
        if self.intro_audio:
            self.intro_channel.stop()
        if self.intro_cap:
            self.intro_cap.release()
        self.intro_cap = None

        self.brand_video_frame.place_forget()
        self.create_start_frame_gui()
        self.start_frame.place(x=0, y=0, width=800, height=600)

        self.play_background_music(volume=0.2)

    # --------------------------------------------------------------------------
    # Pantalla inicial
    # --------------------------------------------------------------------------
    def create_start_frame_gui(self):
        """Construye la pantalla inicial de la app (botones Login y SignUp)."""
        if hasattr(self, "start_content_created") and self.start_content_created:
            return
        self.start_content_created = True

        try:
            bg_img = Image.open("./assets/images/background.png")
            self.bg_photo = ImageTk.PhotoImage(bg_img)
            bg_label = tk.Label(self.start_frame, image=self.bg_photo)
            bg_label.place(x=0, y=0, relwidth=1, relheight=1)
        except:
            pass

        title_label = tk.Label(
            self.start_frame,
            text="Bienvenido al Sistema de Reconocimiento Facial",
            font=("Helvetica", 18, "bold"),
            fg="#ffffff",
            bg="#3d7a9e"
        )
        title_label.place(relx=0.5, rely=0.2, anchor="center")

        try:
            img_login_btn = Image.open("./assets/images/login1.png")
            img_signup_btn = Image.open("./assets/images/register1.png")
            img_login_btn = img_login_btn.resize((180, 190), Image.Resampling.LANCZOS)
            img_signup_btn = img_signup_btn.resize((180, 190), Image.Resampling.LANCZOS)

            self.img_login_btn = ImageTk.PhotoImage(img_login_btn)
            self.img_signup_btn = ImageTk.PhotoImage(img_signup_btn)

            btn_login = tk.Button(
                self.start_frame,
                image=self.img_login_btn,
                command=self.go_to_login,
                bg="#3d7a9e",
                bd=0,
                activebackground="#3d7a9e",
                relief="flat"
            )
            btn_login.place(relx=0.2, rely=0.5, anchor="center")

            btn_signup = tk.Button(
                self.start_frame,
                image=self.img_signup_btn,
                command=self.go_to_signup,
                bg="#3d7a9e",
                bd=0,
                activebackground="#3d7a9e",
                relief="flat"
            )
            btn_signup.place(relx=0.8, rely=0.5, anchor="center")
        except:
            # Si no hay imágenes, muestra texto
            btn_login = tk.Button(
                self.start_frame,
                text="Log In",
                command=self.go_to_login,
                font=("Helvetica", 14),
                bg="#00aaff",
                fg="#ffffff",
                bd=0,
                relief="flat",
                padx=20,
                pady=10
            )
            btn_login.place(relx=0.2, rely=0.5, anchor="center")

            btn_signup = tk.Button(
                self.start_frame,
                text="Sign Up",
                command=self.go_to_signup,
                font=("Helvetica", 14),
                bg="#00aaff",
                fg="#ffffff",
                bd=0,
                relief="flat",
                padx=20,
                pady=10
            )
            btn_signup.place(relx=0.8, rely=0.5, anchor="center")

    def initialize_camera(self):
        """Inicializa la cámara y crea los objetos de Mediapipe."""
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        if not self.cap.isOpened():
            self.camera_ready = False
            return

        mp_face_mesh_module = mp.solutions.face_mesh
        self.face_mesh = mp_face_mesh_module.FaceMesh(
            static_image_mode=False,
            max_num_faces=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

        mp_face_detection = mp.solutions.face_detection
        self.face_detection = mp_face_detection.FaceDetection(
            model_selection=0,
            min_detection_confidence=0.5
        )

        self.camera_ready = True

    # ---------------------------------------------------------------
    # SIGNUP
    # ---------------------------------------------------------------
    def go_to_signup(self):
        """Cambia a la pantalla de registro (SignUp)."""
        self.blink_count = 0
        self.start_frame.place_forget()
        self.setup_signup_frame()
        self.signup_frame.place(x=0, y=0, width=800, height=600)

    def setup_signup_frame(self):
        """Construye el contenido de la pantalla de registro (SignUp)."""
        if hasattr(self, 'signup_content_created') and self.signup_content_created:
            return
        self.signup_content_created = True

        lbl_title = tk.Label(
            self.signup_frame,
            text="Register",
            font=("Helvetica", 16),
            fg="#ffffff",
            bg="#1e1e1e"
        )
        lbl_title.pack(pady=10)

        frm_inner = tk.Frame(self.signup_frame, bg="#1e1e1e")
        frm_inner.pack(pady=20)

        tk.Label(
            frm_inner,
            text="Ingrese su nombre de usuario:",
            font=("Helvetica", 12),
            fg="#ffffff",
            bg="#1e1e1e"
        ).grid(row=0, column=0, padx=10, pady=10)

        self.username_entry = tk.Entry(frm_inner, font=("Helvetica", 12))
        self.username_entry.grid(row=1, column=0, padx=10, pady=5)

        tk.Button(
            frm_inner,
            text="Aceptar",
            command=self.start_signup,
            font=("Helvetica", 12),
            bg="#00aaff",
            fg="#ffffff",
            bd=0,
            relief="flat",
            padx=10,
            pady=5
        ).grid(row=2, column=0, padx=10, pady=10)

        self.btn_cancelar_signup = tk.Button(
            self.signup_frame,
            text="Cancelar",
            font=("Helvetica", 12),
            bg="#ff6666",
            fg="#ffffff",
            command=self.cancel_signup
        )
        self.btn_cancelar_signup.pack(side="bottom", pady=10)

    def start_signup(self):
        """Valida nombre de usuario y prepara la cámara para el proceso de SignUp."""
        username = self.username_entry.get().strip()
        if username == "":
            messagebox.showerror("Error", "Por favor, ingrese un nombre de usuario.")
            return

        self.username = username
        if os.path.exists(f'usuarios/{self.username}'):
            messagebox.showerror("Error", f"El usuario '{self.username}' ya existe. Elija otro.")
            return
        else:
            os.makedirs(f'usuarios/{self.username}')

        self.in_signup_process = True
        self.current_ellipse_index = 0
        self.captured_count = 0
        self.stable_start_time = None
        self.ellipse_defined_for_this_offset = False

        self.loader_label.place(in_=self.signup_frame, relx=0.5, rely=0.5, anchor="center")
        self.animate_loader()
        self.signup_start_time = time.time()
        self.check_camera_ready_signup()

    def check_camera_ready_signup(self):
        """Revisa si la cámara está lista para iniciar SignUp."""
        if self.camera_ready:
            self.loader_label.place_forget()
            self.start_signup_camera()
        else:
            if time.time() - self.signup_start_time > 60:
                messagebox.showerror("Error", "No se pudo acceder a la cámara (time-out).")
                self.return_to_start()
                return
            else:
                self.after(100, self.check_camera_ready_signup)

    def start_signup_camera(self):
        """Inicia la cámara en la pantalla de registro."""
        self.signup_frame.place(x=0, y=0, width=800, height=600)
        self.video_label.place(x=0, y=0, width=800, height=600, in_=self.signup_frame)

        # Evitar solapamiento: si ya suena algo, no reproducir
        if (not self.channel_instructions.get_busy()) and (not self.channel_errors.get_busy()):
            self.pause_background_music()
            self.channel_instructions.play(self.audio_creador)
            dur = self.audio_creador.get_length()
            self.after(int(dur*1000), self.resume_background_music)

        self.update_frame_signup()

    def cancel_signup(self):
        """Cancela el registro y elimina la carpeta del usuario si existe."""
        if self.username:
            user_folder = f"usuarios/{self.username}"
            if os.path.exists(user_folder):
                try:
                    shutil.rmtree(user_folder)
                except Exception as e:
                    print(f"Error al eliminar carpeta: {e}")
        self.username = ""
        self.in_signup_process = False
        self.return_to_start()

    def update_frame_signup(self):
        """Loop de lectura de cámara para SignUp."""
        if (self.cap is None) or (self.captured_count >= self.max_images):
            return

        ret, frame = self.cap.read()
        if not ret:
            self.video_label.after(10, self.update_frame_signup)
            return

        frame = cv2.flip(frame, 1)
        frame = self.resize_for_label(frame, (800, 600))
        h, w, _ = frame.shape

        # Detección de cara (Mediapipe Face Detection)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detection_results = self.face_detection.process(rgb_frame)

        one_face = False
        face_box = None

        if detection_results.detections and len(detection_results.detections) == 1:
            one_face = True
            detection = detection_results.detections[0]
            box = detection.location_data.relative_bounding_box
            x_min = int(box.xmin * w)
            y_min = int(box.ymin * h)
            x_max = x_min + int(box.width * w)
            y_max = y_min + int(box.height * h)
            face_box = (x_min, y_min, x_max, y_max)

        # YOLO gafas/máscara
        found_glasses_mask = False
        boxes_gm = []
        if one_face and face_box is not None:
            fx1, fy1, fx2, fy2 = face_box
            fx1 = max(0, fx1); fy1 = max(0, fy1)
            fx2 = min(w, fx2); fy2 = min(h, fy2)
            face_roi = frame[fy1:fy2, fx1:fx2]
            if face_roi.size > 0:
                face_640 = cv2.resize(face_roi, (640, 640))
                results = self.yolo_model.predict(face_640, imgsz=640, conf=0.25, verbose=False)
                for r in results:
                    for box_yolo in r.boxes:
                        cls_id = int(box_yolo.cls[0])
                        class_name = r.names[cls_id].lower()
                        if class_name not in ["glasses", "mask"]:
                            continue
                        found_glasses_mask = True

                        x1_640, y1_640 = int(box_yolo.xyxy[0][0]), int(box_yolo.xyxy[0][1])
                        x2_640, y2_640 = int(box_yolo.xyxy[0][2]), int(box_yolo.xyxy[0][3])
                        face_w = fx2 - fx1
                        face_h = fy2 - fy1
                        scale_x = face_w / 640.0
                        scale_y = face_h / 640.0

                        bx1 = fx1 + int(x1_640 * scale_x)
                        by1 = fy1 + int(y1_640 * scale_y)
                        bx2 = fx1 + int(x2_640 * scale_x)
                        by2 = fy1 + int(y2_640 * scale_y)
                        boxes_gm.append((bx1, by1, bx2, by2, class_name))



        # Dibujar boxes gafas/máscara
        for (x1_gm, y1_gm, x2_gm, y2_gm, cname) in boxes_gm:
            color = (0, 255, 255) if cname == "glasses" else (255, 0, 255)
            cv2.rectangle(frame, (x1_gm, y1_gm), (x2_gm, y2_gm), color, 2)
            frame = put_text_pil(frame, cname, (x1_gm, y1_gm - 35), font_size=28, color=color)


        # Mensajes si gafas/máscara o spoof
        font_scale_cv = 1.3
        if found_glasses_mask:
            cv2.putText(
                frame,
                "Debe quitarse gafas/mascara",
                (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale_cv,
                (0, 0, 255),
                2
            )

        if found_glasses_mask:
            self.show_frame_on_label(frame)
            self.video_label.after(10, self.update_frame_signup)
            return

        # Analizar FaceMesh para:
        # - Head pose
        # - Saber si la cara está en el óvalo con la malla (mesh points)
        mesh_results = self.face_mesh.process(rgb_frame)
        head_pose_text = "Undefined"
        head_is_forward = False
        head_pose_color = (255, 0, 0)  # color por defecto

        if mesh_results.multi_face_landmarks and len(mesh_results.multi_face_landmarks) == 1:
            face_landmarks = mesh_results.multi_face_landmarks[0]
            face_3d = []
            face_2d = []
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in [33, 263, 1, 61, 291, 199]:
                    xx, yy = int(lm.x * w), int(lm.y * h)
                    face_2d.append([xx, yy])
                    face_3d.append([xx, yy, lm.z])
            if len(face_2d) == 6:
                focal_length = w
                cam_matrix = np.array([[focal_length,0,h/2],
                                       [0,focal_length,w/2],
                                       [0,0,1]])
                dist_matrix = np.zeros((4,1), dtype=np.float64)
                success_pnp, rot_vec, trans_vec = cv2.solvePnP(
                    np.array(face_3d, dtype=np.float64),
                    np.array(face_2d, dtype=np.float64),
                    cam_matrix,
                    dist_matrix
                )
                if success_pnp:
                    rmat, _ = cv2.Rodrigues(rot_vec)
                    angles, _, _, _, _, _ = cv2.RQDecomp3x3(rmat)
                    x_angle = angles[0]*360
                    y_angle = angles[1]*360
                    if y_angle < -4:
                        head_pose_text = "Mirando Izq."
                    elif y_angle > 5:
                        head_pose_text = "Mirando Der."
                    elif x_angle < -3:
                        head_pose_text = "Mirando Abajo"
                    elif x_angle > 8:
                        head_pose_text = "Mirando Arriba"
                    else:
                        head_pose_text = "Frente"
                        head_is_forward = True

        # Determinar color de la pose
        head_pose_color = (0, 255, 0) if head_is_forward else (255, 0, 0)
        # Mostrar texto de la pose
        head_full_text = f"Pos. Cabeza: {head_pose_text}"
        frame = put_text_pil(frame, head_full_text, (30, 100), font_size=28, color=head_pose_color)

        # Óvalo + ojos y boca
        if one_face and mesh_results.multi_face_landmarks and len(mesh_results.multi_face_landmarks) == 1:
            if not self.ellipse_defined_for_this_offset:
                self.define_ellipse_for_current_offset(face_box, (w,h))
                self.ellipse_defined_for_this_offset = True

            # Calculamos 4 puntos (top, bottom, left, right) de la mesh
            # para verificar con is_face_inside_rotated_ellipse_mesh
            face_landmarks = mesh_results.multi_face_landmarks[0]
            xs = []
            ys = []
            for lm in face_landmarks.landmark:
                xs.append(int(lm.x * w))
                ys.append(int(lm.y * h))

            x_left   = min(xs)
            x_right  = max(xs)
            y_top    = min(ys)
            y_bottom = max(ys)

            pt_top    = ((x_left + x_right)//2, y_top)
            pt_bottom = ((x_left + x_right)//2, y_bottom)
            pt_left   = (x_left, (y_top + y_bottom)//2)
            pt_right  = (x_right, (y_top + y_bottom)//2)
            mesh_points = [pt_top, pt_bottom, pt_left, pt_right]

            cx, cy, rx, ry = self.current_ellipse
            face_inside = self.is_face_inside_rotated_ellipse_mesh(
                mesh_points, 
                cx, 
                cy, 
                rx, 
                ry, 
                self.ellipse_angle
            )

            # Comprobamos ojos y boca
            eyes_open = False
            mouth_closed = False

            if head_is_forward:
                landmarks = face_landmarks.landmark
                left_eye_indices = [33,160,158,133,153,144]
                right_eye_indices= [362,385,387,263,373,380]
                left_eye_coords = [(int(landmarks[i].x*w), int(landmarks[i].y*h)) for i in left_eye_indices]
                right_eye_coords= [(int(landmarks[i].x*w),int(landmarks[i].y*h)) for i in right_eye_indices]
                left_EAR = self.calculate_EAR(left_eye_coords)
                right_EAR= self.calculate_EAR(right_eye_coords)
                EAR_thr = 0.2
                if (left_EAR>=EAR_thr) and (right_EAR>=EAR_thr):
                    eyes_open=True

                # Boca
                mt_idx, mb_idx = 13, 14
                mt = (int(landmarks[mt_idx].x*w), int(landmarks[mt_idx].y*h))
                mb = (int(landmarks[mb_idx].x*w), int(landmarks[mb_idx].y*h))
                dist_mouth = np.linalg.norm(np.array(mt)-np.array(mb))
                mouth_closed = (dist_mouth<=30)

            conditions_ok = (face_inside and eyes_open and mouth_closed and head_is_forward)
            ellipse_color = (0,0,255)

            if conditions_ok:
                ellipse_color = (0,255,0)
                if self.stable_start_time is None:
                    self.stable_start_time = time.time()
                stable_time = time.time() - self.stable_start_time

                msg_estable = f"Estable: {stable_time:.1f} seg"
                cv2.putText(
                    frame,
                    msg_estable,
                    (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.3,
                    (0, 255, 0),
                    2
                )

                if stable_time >= self.required_stable_seconds:
                    # Capturamos foto
                    if self.channel_instructions.get_busy():
                        self.channel_instructions.stop()
                    if self.channel_errors.get_busy():
                        self.channel_errors.stop()

                    self.capture_and_crop_face(face_box, True)
                    self.show_flash_effect()
                    self.pause_background_music()
                    self.camera_sound.play()
                    dur_cam = self.camera_sound.get_length()
                    self.after(int(dur_cam*1000), self.resume_background_music)

                    self.current_ellipse_index += 1
                    self.captured_count += 1
                    self.ellipse_defined_for_this_offset = False
                    self.stable_start_time = None

                    if (self.captured_count>=self.max_images or
                        self.current_ellipse_index>=self.total_ellipses):
                        self.finish_signup_captures()
                        return
            else:
                # Mostrar razones
                reasons=[]
                if not face_inside: reasons.append("Cara fuera del ovalo (mesh)")
                if not eyes_open: reasons.append("Ojos cerrados")
                if not mouth_closed: reasons.append("Boca abierta")
                if not head_is_forward: reasons.append("Cabeza no de frente")

                if reasons:
                    msg_err = "Condiciones no cumplidas:"
                    cv2.putText(
                        frame,
                        msg_err,
                        (int(w/2), 40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.9,
                        (0, 0, 255),
                        2
                    )
                    y_offset = 70
                    for reason in reasons:
                        cv2.putText(
                            frame,
                            reason,
                            (int(w/2 + 20), y_offset),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.9,
                            (0, 0, 255),
                            2
                        )
                        y_offset += 30
                self.handle_error_sounds(eyes_open, mouth_closed)
                self.stable_start_time=None

            cv2.ellipse(frame,(cx,cy),(rx,ry),self.ellipse_angle,0,360,ellipse_color,2)

        else:
            # Mensaje "No se detecta ninguna cara..."
            cv2.putText(
                frame,
                "No se detecta ninguna cara...",
                (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.1,
                (0,0,255),
                2
            )

        # Dibujar FaceMesh en pantalla
        if mesh_results.multi_face_landmarks:
            for face_landmarks in mesh_results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec
                )

        self.show_frame_on_label(frame)
        self.video_label.after(10, self.update_frame_signup)

    def capture_and_crop_face(self, face_box, is_signup=False):
        """Captura frame actual, recorta cara y guarda si is_signup=True."""
        ret, raw_frame = self.cap.read()
        if not ret:
            return
        raw_frame = cv2.flip(raw_frame,1)
        raw_frame = self.resize_for_label(raw_frame,(800,600))

        x_min, y_min, x_max, y_max = face_box
        hh, ww, _ = raw_frame.shape
        x_min = max(0,x_min); y_min = max(0,y_min)
        x_max = min(ww,x_max); y_max = min(hh,y_max)
        face_unprocessed = raw_frame[y_min:y_max, x_min:x_max]
        if face_unprocessed.size==0:
            return
        

        face_processed = enhance_image_cv2(face_unprocessed)

        if is_signup:
            img_count = self.captured_count+1
            base_path = f"usuarios/{self.username}"
            unp_path = os.path.join(base_path,f"foto_{img_count}_unproc.png")
            pro_path = os.path.join(base_path,f"foto_{img_count}.png")

            pil_unproc = Image.fromarray(cv2.cvtColor(face_unprocessed,cv2.COLOR_BGR2RGB))
            pil_unproc.save(unp_path)
            pil_proc = Image.fromarray(cv2.cvtColor(face_processed,cv2.COLOR_BGR2RGB))
            pil_proc.save(pro_path)

    def finish_signup_captures(self):
        """Genera embeddings y finaliza signup."""
        self.generate_embeddings()
        messagebox.showinfo(
            "Sign Up",
            f"Se han capturado {self.captured_count} imágenes. Registro completado para {self.username}!"
        )
        self.on_signup_complete()

    def generate_embeddings(self):
        """Genera embeddings Facenet de las fotos capturadas en SignUp."""
        user_folder = f"usuarios/{self.username}"
        embeddings_list = []
        for i in range(1,self.captured_count+1):
            img_path = os.path.join(user_folder,f"foto_{i}.png")
            if os.path.exists(img_path):
                emb_data = DeepFace.represent(img_path=img_path,model_name="Facenet",enforce_detection=False)
                if(len(emb_data)>0)and("embedding" in emb_data[0]):
                    arr = np.array(emb_data[0]["embedding"]).flatten()
                    if arr.shape[0]==128:
                        embeddings_list.append(arr.tolist())
        data={"username":self.username,"embeddings":embeddings_list}
        with open(os.path.join(user_folder,"embeddings.json"),"w") as f:
            json.dump(data,f,indent=2)

    def on_signup_complete(self):
        """Retorna a start tras completar signup."""
        self.username=""
        self.in_signup_process=False
        self.video_label.place_forget()
        self.return_to_start()

    # ---------------------------------------------------------------
    # LOGIN
    # ---------------------------------------------------------------
    def go_to_login(self):
        """Cambia a la pantalla de Login."""
        self.blink_count=0
        self.start_frame.place_forget()
        self.setup_login_frame()
        self.login_frame.place(x=0, y=0, width=800, height=600)

        self.login_timeout_start=time.time()
        self.blink_count=0
        self.prev_eye_state="open"
        self.eye_transition_flag=False
        self.login_image_captured=None
        self.login_embedding_vector=None
        self.current_logged_in_user=None
        self.login_image_pil=None
        self.login_image_pil_unprocessed=None

        self.loader_label.place(in_=self.login_frame, relx=0.5, rely=0.5, anchor="center")
        self.animate_loader()
        self.check_camera_ready_login()

    def setup_login_frame(self):
        if hasattr(self,'login_content_created') and self.login_content_created:
            return
        self.login_content_created=True

        lbl_title=tk.Label(self.login_frame,text="Login",font=("Helvetica",16),
                           fg="#ffffff",bg="#1e1e1e")
        lbl_title.pack(pady=10)

        btn_volver=tk.Button(self.login_frame,text="Volver",
                             font=("Helvetica",12),bg="#ff6666",fg="#ffffff",
                             command=self.return_to_start)
        btn_volver.place(relx=0.5,rely=1.0,anchor="s",y=-10,in_=self.login_frame)

    def check_camera_ready_login(self):
        """Revisa cámara para login."""
        if self.camera_ready:
            self.loader_label.place_forget()
            self.video_label.place(x=0,y=0,width=800,height=600,in_=self.login_frame)
            self.start_login_process()
        else:
            if time.time()-self.login_timeout_start>60:
                self.loader_label.place_forget()
                messagebox.showerror("Error","No se pudo acceder a la cámara (time-out).")
                self.return_to_start()
                return
            self.after(100,self.check_camera_ready_login)

    def start_login_process(self):
        """Inicia la captura de cámara para login."""
        self.is_logging_in=True
        self.login_start_time=time.time()

        # Evitar solapamiento de audio
        if (not self.channel_instructions.get_busy()) and (not self.channel_errors.get_busy()):
            self.pause_background_music()
            self.channel_instructions.play(self.audio_login)
            dur = self.audio_login.get_length()
            self.after(int(dur*1000), self.resume_background_music)

        self.update_frame_login()

    def update_frame_login(self):
        """Loop de cámara para login."""
        if not self.is_logging_in or (self.cap is None):
            return

        ret,frame=self.cap.read()
        if not ret:
            self.video_label.after(10,self.update_frame_login)
            return

        clean_frame=frame.copy()
        frame=cv2.flip(frame,1)
        clean_frame=cv2.flip(clean_frame,1)

        frame=self.resize_for_label(frame,(800,600))
        clean_frame=self.resize_for_label(clean_frame,(800,600))
        h,w,_=frame.shape

        font_scale_cv = 1.3

        # Spoofing
        frame_640=cv2.resize(frame,(640,640))
        results_spoof=self.yolo_model2.predict(frame_640,imgsz=640,conf=0.25,verbose=False)

        boxes_spoof=[]
        found_restriction=False
        class_name2=""  # para mostrar texto

        for r2 in results_spoof:
            for box2 in r2.boxes:
                cls2_id=int(box2.cls[0])
                class_name2=r2.names[cls2_id]
                if class_name2 in["Device","Masks","Photo"]:
                    x1_640,y1_640=int(box2.xyxy[0][0]),int(box2.xyxy[0][1])
                    x2_640,y2_640=int(box2.xyxy[0][2]),int(box2.xyxy[0][3])
                    scale_x=w/640.0
                    scale_y=h/640.0
                    x1=int(x1_640*scale_x)
                    y1=int(y1_640*scale_y)
                    x2=int(x2_640*scale_x)
                    y2=int(y2_640*scale_y)
                    boxes_spoof.append((x1,y1,x2,y2))

        if boxes_spoof:
            self.spoof_lost_count=0
            self.last_spoof_box=boxes_spoof[0]
        else:
            if self.last_spoof_box is not None:
                self.spoof_lost_count+=1
                if self.spoof_lost_count<self.spoof_lost_threshold:
                    boxes_spoof.append(self.last_spoof_box)
                else:
                    self.last_spoof_box=None

        for(sx1,sy1,sx2,sy2)in boxes_spoof:
            cv2.rectangle(frame,(sx1,sy1),(sx2,sy2),(0,0,255),2)
            cv2.putText(
                frame,
                "Spoofing Detectado",
                (sx2,sy1),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale_cv,
                (0,0,255),
                2
            )
            found_restriction=True

        # Mediapipe face detection
        rgb_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        detection_results=self.face_detection.process(rgb_frame)

        # YOLO gafas/máscara
        if detection_results.detections:
            if len(detection_results.detections)>1:
                cv2.putText(
                    frame,
                    "Debe haber 1 sola cara",
                    (30,90),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale_cv,
                    (0,0,255),
                    2
                )
                found_restriction=True
            else:
                detection=detection_results.detections[0]
                box=detection.location_data.relative_bounding_box
                x_min=int(box.xmin*w)
                y_min=int(box.ymin*h)
                x_max=x_min+int(box.width*w)
                y_max=y_min+int(box.height*h)
                fx1=max(0,x_min);fy1=max(0,y_min)
                fx2=min(w,x_max);fy2=min(h,y_max)

                face_roi=frame[fy1:fy2, fx1:fx2]
                face_w=fx2-fx1
                face_h=fy2-fy1
                if face_roi.size>0:
                    face_640=cv2.resize(face_roi,(640,640))
                    results_gafas_masc=self.yolo_model.predict(face_640,imgsz=640,conf=0.25,verbose=False)
                    for r in results_gafas_masc:
                        for box_yolo in r.boxes:
                            cls_id=int(box_yolo.cls[0])
                            class_name=r.names[cls_id].lower()
                            if class_name not in["glasses","mask"]:
                                continue
                            found_restriction=True
                            x1_640,y1_640=int(box_yolo.xyxy[0][0]),int(box_yolo.xyxy[0][1])
                            x2_640,y2_640=int(box_yolo.xyxy[0][2]),int(box_yolo.xyxy[0][3])
                            s_x=face_w/640.0
                            s_y=face_h/640.0
                            bx1=fx1+int(x1_640*s_x)
                            by1=fy1+int(y1_640*s_y)
                            bx2=fx1+int(x2_640*s_x)
                            by2=fy1+int(y2_640*s_y)
                            color=(0,255,255) if class_name=="glasses" else (255,0,255)
                            cv2.rectangle(frame,(bx1,by1),(bx2,by2),color,2)
                            cv2.putText(
                                frame,
                                class_name,
                                (bx2,by1),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                font_scale_cv,
                                color,
                                2
                            )
                            msg_login = "Debe quitarse las gafas" if class_name=="glasses" else "Debe quitarse la mascarilla"
                            cv2.putText(
                                frame,
                                msg_login,
                                (30,160),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                font_scale_cv,
                                (0,0,255),
                                2
                            )

        # Head pose
        mesh_results_2=self.face_mesh.process(rgb_frame)
        head_pose_text="Undefined"
        head_is_forward=False
        color2 = (255,0,0)  ### CORREGIDO: Asignar por defecto

        if mesh_results_2.multi_face_landmarks and len(mesh_results_2.multi_face_landmarks)==1:
            face_landmarks=mesh_results_2.multi_face_landmarks[0]
            face_3d=[]
            face_2d=[]
            for idx,lm in enumerate(face_landmarks.landmark):
                if idx in [33,263,1,61,291,199]:
                    xx,yy=int(lm.x*w),int(lm.y*h)
                    face_2d.append([xx,yy])
                    face_3d.append([xx,yy,lm.z])
            if len(face_2d)==6:
                focal_length=w
                cam_matrix=np.array([[focal_length,0,h/2],
                                     [0,focal_length,w/2],
                                     [0,0,1]])
                dist_matrix=np.zeros((4,1),dtype=np.float64)
                ok_pnp, rot_vec, trans_vec=cv2.solvePnP(
                    np.array(face_3d,dtype=np.float64),
                    np.array(face_2d,dtype=np.float64),
                    cam_matrix, dist_matrix
                )
                if ok_pnp:
                    rmat,_=cv2.Rodrigues(rot_vec)
                    angles,_,_,_,_,_=cv2.RQDecomp3x3(rmat)
                    x_angle=angles[0]*360
                    y_angle=angles[1]*360
                    if y_angle< -4:
                        head_pose_text="Mirando Izq."
                    elif y_angle>5:
                        head_pose_text="Mirando Der."
                    elif x_angle< -3:
                        head_pose_text="Mirando Abajo"
                    elif x_angle>8:
                        head_pose_text="Mirando Arriba"
                    else:
                        head_pose_text="Frente"
                        head_is_forward=True

        # Elegir color
        if head_is_forward:
            color2=(0,255,0)
        else:
            color2=(255,0,0)

        head_pose_text = "Pos. Cabeza: " + head_pose_text
        cv2.putText(
            frame,
            head_pose_text,
            (30,100),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale_cv,
            color2,
            2
        )

        if not head_is_forward:
            found_restriction=True
            cv2.putText(
                frame,
                "La cabeza debe mirar al frente",
                (30,40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.1,
                (0,0,255),
                2
            )

        if found_restriction:
            self.blink_count=0
            self.show_frame_on_label(frame)
            self.video_label.after(10,self.update_frame_login)
            return

        # Revisar detección normal
        if not detection_results.detections:
            cv2.putText(
                frame,
                "No se detecta rostro",
                (30,90),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale_cv,
                (0,0,255),
                2
            )
            self.show_frame_on_label(frame)
            self.video_label.after(10,self.update_frame_login)
            return

        if len(detection_results.detections)>1:
            cv2.putText(
                frame,
                "Debe haber 1 sola cara",
                (30,90),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale_cv,
                (0,0,255),
                2
            )
            self.show_frame_on_label(frame)
            self.video_label.after(10,self.update_frame_login)
            return

        detection=detection_results.detections[0]
        if not mesh_results_2.multi_face_landmarks:
            self.show_frame_on_label(frame)
            self.video_label.after(10,self.update_frame_login)
            return

        face_landmarks_2=mesh_results_2.multi_face_landmarks[0]
        landmarks=face_landmarks_2.landmark
        left_eye_indices=[33,160,158,133,153,144]
        right_eye_indices=[362,385,387,263,373,380]
        left_eye_coords=[(int(landmarks[i].x*w),int(landmarks[i].y*h)) for i in left_eye_indices]
        right_eye_coords=[(int(landmarks[i].x*w),int(landmarks[i].y*h)) for i in right_eye_indices]
        left_EAR=self.calculate_EAR(left_eye_coords)
        right_EAR=self.calculate_EAR(right_eye_coords)
        ear_thr=0.2
        eyes_open_now=(left_EAR>=ear_thr)and(right_EAR>=ear_thr)

        if eyes_open_now:
            if self.prev_eye_state=="closed" and not self.eye_transition_flag:
                self.blink_count+=1
                self.eye_transition_flag=True
            self.prev_eye_state="open"
        else:
            if self.prev_eye_state=="open":
                self.eye_transition_flag=False
            self.prev_eye_state="closed"

        cv2.putText(
            frame,
            f"Pestaneos: {self.blink_count}",
            (30, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.3,
            (0,255,0),
            2
        )

        if self.blink_count>=3:
            mouth_top_idx=13
            mouth_bottom_idx=14
            mt=(int(landmarks[mouth_top_idx].x*w), int(landmarks[mouth_top_idx].y*h))
            mb=(int(landmarks[mouth_bottom_idx].x*w), int(landmarks[mouth_bottom_idx].y*h))
            mouth_dist=np.linalg.norm(np.array(mt)-np.array(mb))
            mouth_closed=(mouth_dist<=30)
            if eyes_open_now and mouth_closed:
                # Evitar solapamiento de audio
                if self.channel_instructions.get_busy():
                    self.channel_instructions.stop()
                if self.channel_errors.get_busy():
                    self.channel_errors.stop()

                face_crop=self.take_login_picture_rotated(clean_frame,detection)
                if face_crop is None:
                    cv2.putText(
                        frame,
                        "No se pudo recortar la cara rotada",
                        (30,420),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.1,
                        (0,0,255),
                        2
                    )
                return
            else:
                faltan=[]
                if not eyes_open_now:
                    faltan.append("ojos abiertos")
                if not mouth_closed:
                    faltan.append("boca cerrada")
                if faltan:
                    msg_f="Condición faltante: "+", ".join(faltan)
                    cv2.putText(
                        frame,
                        msg_f,
                        (30,420),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1.1,
                        (0,0,255),
                        2
                    )

        self.show_frame_on_label(frame)
        self.video_label.after(10,self.update_frame_login)

    def take_login_picture_rotated(self,frame_bgr,detection):
        """Rota la imagen para alinear ojos y recorta la cara. Luego determina el usuario."""
        keypoints=detection.location_data.relative_keypoints
        if len(keypoints)<2:
            self.finish_login_process(open_profile=True)
            return None

        hh, ww, _=frame_bgr.shape
        right_eye_kp=keypoints[0]
        left_eye_kp=keypoints[1]
        rex=int(right_eye_kp.x*ww)
        rey=int(right_eye_kp.y*hh)
        lex=int(left_eye_kp.x*ww)
        ley=int(left_eye_kp.y*hh)

        dx=lex-rex
        dy=ley-rey
        angle=np.degrees(np.arctan2(dy,dx))

        cx=(lex+rex)//2
        cy=(ley+rey)//2
        M=cv2.getRotationMatrix2D((cx,cy),angle,1.0)
        rotated=cv2.warpAffine(frame_bgr,M,(ww,hh))

        box=detection.location_data.relative_bounding_box
        fxmin=int(box.xmin*ww);fymin=int(box.ymin*hh)
        fw=int(box.width*ww);fh=int(box.height*hh)
        fxmax=fxmin+fw
        fymax=fymin+fh

        corners=[(fxmin,fymin),(fxmax,fymin),(fxmin,fymax),(fxmax,fymax)]
        rot_corners=[]
        for(ccx,ccy)in corners:
            new_xy=np.dot(M,np.array([ccx,ccy,1]))
            rot_corners.append((int(new_xy[0]),int(new_xy[1])))

        xs=[p[0] for p in rot_corners]
        ys=[p[1] for p in rot_corners]
        xmin_r,xmax_r=min(xs),max(xs)
        ymin_r,ymax_r=min(ys),max(ys)

        margin_x=int(0.10*fw)
        margin_y=int(0.20*fh)

        xmin_r=max(0,xmin_r-margin_x)
        xmax_r=min(ww,xmax_r+margin_x)
        ymin_r=max(0,ymin_r-margin_y)
        ymax_r=min(hh,ymax_r+margin_y)

        if xmin_r>=xmax_r or ymin_r>=ymax_r:
            self.finish_login_process(open_profile=True)
            return None

        face_unprocessed=rotated[ymin_r:ymax_r,xmin_r:xmax_r]
        if face_unprocessed.size==0:
            self.finish_login_process(open_profile=True)
            return None

        face_processed=enhance_image_cv2(face_unprocessed)
        self.login_image_pil_unprocessed=Image.fromarray(cv2.cvtColor(face_unprocessed,cv2.COLOR_BGR2RGB))
        self.login_image_pil=Image.fromarray(cv2.cvtColor(face_processed,cv2.COLOR_BGR2RGB))

        temp_path="login_temp.png"
        self.login_image_pil.save(temp_path)

        emb_data=DeepFace.represent(img_path=temp_path,model_name="Facenet",enforce_detection=False)
        if(len(emb_data)==0)or("embedding"not in emb_data[0]):
            messagebox.showerror("Error","No se pudo generar embedding.")
            self.current_logged_in_user="Desconocido"
            self.login_embedding_vector=None
            self.finish_login_process(open_profile=True)
            if os.path.exists(temp_path): os.remove(temp_path)
            return None

        arr=np.array(emb_data[0]["embedding"]).flatten()
        if arr.shape[0]!=128:
            messagebox.showerror("Error","Embedding inválido (no 128).")
            self.current_logged_in_user="Desconocido"
            self.login_embedding_vector=None
            self.finish_login_process(open_profile=True)
            if os.path.exists(temp_path):os.remove(temp_path)
            return None

        user_dirs=[d for d in os.listdir("usuarios/") if os.path.isdir(os.path.join("usuarios/", d))]
        if not user_dirs:
            messagebox.showinfo("Login","No hay usuarios registrados. Serás 'Desconocido'.")
            self.current_logged_in_user="Desconocido"
            self.login_embedding_vector=arr
            self.finish_login_process(open_profile=True)
            if os.path.exists(temp_path):os.remove(temp_path)
            return face_unprocessed

        threshold=0.25
        best_distance=float('inf')
        best_user=None

        for user_dir in user_dirs:
            emb_json_path=os.path.join("usuarios",user_dir,"embeddings.json")
            if not os.path.exists(emb_json_path): 
                continue
            with open(emb_json_path,"r")as f:
                data=json.load(f)
            un=data["username"]
            user_embs=data["embeddings"]
            for emb_vector in user_embs:
                user_arr=np.array(emb_vector).flatten()
                if user_arr.shape[0]!=128: 
                    continue
                dist=df_verification.find_cosine_distance(arr,user_arr)
                if dist<best_distance:
                    best_distance=dist
                    best_user=un

        if os.path.exists(temp_path):
            os.remove(temp_path)

        if (best_user is not None)and(best_distance<threshold):
            messagebox.showinfo("Login",f"¡Bienvenido, {best_user}!")
            self.current_logged_in_user=best_user
            self.login_embedding_vector=arr
        else:
            messagebox.showinfo("Login","No se encontró coincidencia. Serás 'Desconocido'.")
            self.current_logged_in_user="Desconocido"
            self.login_embedding_vector=arr

        self.finish_login_process(open_profile=True)
        return face_unprocessed

    def finish_login_process(self, open_profile=False):
        """Finaliza el proceso de Login y pasa a la pantalla de perfil si open_profile=True."""
        self.is_logging_in=False
        self.video_label.place_forget()

        ### NUEVO: Registrar en el log la sesión
        self.log_user_login(self.current_logged_in_user)

        if open_profile:
            self.show_profile_frame()
        else:
            self.return_to_start()

    ### NUEVO: Función para escribir el log de inicios de sesión
    def log_user_login(self, username):
        """
        Inserta una línea al comienzo del archivo 'login_history.log' 
        con:  <usuario>,<fecha-hora>\n
        """
        log_file = "login_history.log"
        now_str = time.strftime("%Y-%m-%d %H:%M:%S")
        new_line = f"{username},{now_str}\n"
        lines = []
        if os.path.exists(log_file):
            with open(log_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        # Insertar al inicio
        lines.insert(0, new_line)
        with open(log_file, 'w', encoding='utf-8') as f:
            f.writelines(lines)

    # ---------------------------------------------------------------
    # PERFIL
    # ---------------------------------------------------------------
    def show_profile_frame(self):
        self.login_frame.place_forget()
        self.profile_frame.place(x=0,y=0,width=800,height=600)
        self.setup_profile_frame()

    def setup_profile_frame(self):
        """Construye la pantalla de perfil."""
        for w in self.profile_frame.winfo_children():
            w.destroy()

        frame_title=tk.Frame(self.profile_frame,bg="#444444")
        frame_title.pack(fill="x")

        user_label_text=f"Perfil de {self.current_logged_in_user}"
        lbl_title=tk.Label(frame_title,text=user_label_text,font=("Helvetica",18,"bold"),
                           bg="#444444",fg="#ffffff")
        lbl_title.pack(anchor="center",pady=10)

        frame_main=tk.Frame(self.profile_frame,bg="#2c2c2c")
        frame_main.pack(fill="both",expand=True)

        frame_left=tk.Frame(frame_main,bg="#333333")
        frame_left.pack(side="left",fill="y",padx=20,pady=20)

        frame_center=tk.Frame(frame_main,bg="#2c2c2c")
        frame_center.pack(side="left",fill="both",expand=True,pady=20)

        frame_img=tk.LabelFrame(frame_center,text="Comparación",
                                fg="#ffffff",bg="#2c2c2c",labelanchor="n",
                                font=("Helvetica",12,"bold"),bd=2)
        frame_img.pack(side="top",padx=20,pady=10)

        frm_two=tk.Frame(frame_img,bg="#2c2c2c")
        frm_two.pack()

        if self.login_image_pil and self.login_image_pil_unprocessed:
            img_tk_proc=ImageTk.PhotoImage(self.login_image_pil)
            lbl_image_proc=tk.Label(frm_two,image=img_tk_proc,bg="#2c2c2c")
            lbl_image_proc.image=img_tk_proc
            lbl_image_proc.pack(side="left",padx=10,pady=10)

            img_tk_unproc=ImageTk.PhotoImage(self.login_image_pil_unprocessed)
            lbl_image_unproc=tk.Label(frm_two,image=img_tk_unproc,bg="#2c2c2c")
            lbl_image_unproc.image=img_tk_unproc
            lbl_image_unproc.pack(side="right",padx=10,pady=10)
        else:
            lbl_no_img=tk.Label(frame_img,text="(No hay imagen de login)",
                                font=("Helvetica",12),bg="#2c2c2c",fg="#ffffff")
            lbl_no_img.pack(padx=10,pady=10)

        self.predictions_label=tk.Label(frame_center,
                                        textvariable=self.predictions_label_var,
                                        font=("Helvetica",12),
                                        fg="#00FF00",bg="#2c2c2c",justify="left")
        self.predictions_label.pack(side="top",padx=10,pady=10)

        btn_stats=tk.Button(frame_left,text="Ver Estadísticas",
                            font=("Helvetica",12,"bold"),bg="#00aaff",fg="#ffffff",
                            padx=10,pady=5,command=self.show_stats_frame)
        btn_stats.pack(pady=5,fill="x")

        btn_symmetry=tk.Button(frame_left,text="Evaluar Simetría",
                               font=("Helvetica",12,"bold"),bg="#00aaff",fg="#ffffff",
                               padx=10,pady=5,command=self.evaluate_facial_symmetry_window)
        btn_symmetry.pack(pady=5,fill="x")

        btn_preds=tk.Button(frame_left,text="Predicciones",
                            font=("Helvetica",12,"bold"),bg="#00aaff",fg="#ffffff",
                            padx=10,pady=5,command=self.show_predictions)
        btn_preds.pack(pady=5,fill="x")

        lbl_sym=tk.Label(frame_left,textvariable=self.simmetry_label_var,
                         font=("Helvetica",12,"bold"),bg="#333333",fg="#ffffff")
        lbl_sym.pack(pady=10,fill="x")

        # NUEVO: Botón "Cuenta"/"Datos"
        if self.current_logged_in_user and self.current_logged_in_user != "Desconocido":
            btn_account = tk.Button(
                frame_left, 
                text="Cuenta", 
                font=("Helvetica",12,"bold"),
                bg="#90ee90", 
                fg="#000000", 
                padx=10, 
                pady=5,
                command=self.show_account_info
            )
            btn_account.pack(pady=5,fill="x")

        if self.current_logged_in_user=="Desconocido":
            btn_registrar=tk.Button(frame_left,text="Registrar",
                                    font=("Helvetica",12,"bold"),bg="#ffa500",fg="#ffffff",
                                    padx=10,pady=5,command=self.go_to_signup)
            btn_registrar.pack(pady=5,fill="x")

        btn_volver=tk.Button(frame_left,text="Volver",
                             font=("Helvetica",12,"bold"),bg="#ff6666",fg="#ffffff",
                             padx=10,pady=5,command=self.return_to_start)
        btn_volver.pack(side="bottom",pady=20,fill="x")

    ### NUEVO: Mostrar datos de la cuenta desde el log
    def show_account_info(self):
        """
        Lee el archivo 'login_history.log' y muestra:
         - Número de veces que ha iniciado sesión el usuario actual.
         - Fecha/hora de la última conexión (la más reciente en el log).
        """
        if (not self.current_logged_in_user) or (self.current_logged_in_user == "Desconocido"):
            messagebox.showinfo("Cuenta", "No hay datos para 'Desconocido'.")
            return

        log_file = "login_history.log"
        if not os.path.exists(log_file):
            messagebox.showinfo("Cuenta", "No hay historial de inicio de sesión aún.")
            return

        with open(log_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # Filtrar líneas que empiecen con "<username>,"
        user_lines = [line.strip() for line in lines if line.startswith(f"{self.current_logged_in_user},")]
        times_logged_in = len(user_lines)

        #No deberia de suceder, pero...
        if times_logged_in == 0:
            messagebox.showinfo("Cuenta", "No se ha registrado inicio de sesión para este usuario.")
            return
        # Verificar si hay más de una línea para identificar la última conexión previa
        if times_logged_in > 1:
            last_connection = user_lines[1]  # Segunda línea es la conexión previa más reciente
            parts_last_connection = last_connection.split(',')
            date_time_last_connection = parts_last_connection[1] if len(parts_last_connection) > 1 else "desconocido"
        else:
            date_time_last_connection = "No existente"  # No hay conexión previa registrada

        info_msg = (
            f"Has iniciado sesión {times_logged_in} veces.\n"
            f"Última conexión: {date_time_last_connection}"
        )
        messagebox.showinfo("Cuenta", info_msg)

    # ---------------------------------------------------------------
    # Funciones de Perfil, Stats, etc.
    # ---------------------------------------------------------------
    def evaluate_facial_symmetry_window(self):
        # 1. Verificar que tengas la imagen de login
        if self.login_image_pil is None:
            messagebox.showinfo("Simetría","No hay imagen de login procesada.")
            return

        # 2. Convertir a BGR para procesar con Mediapipe
        face_bgr = cv2.cvtColor(np.array(self.login_image_pil), cv2.COLOR_RGB2BGR)

        # 3. Correr FaceMesh sobre esa imagen
        mp_mesh = mp.solutions.face_mesh
        with mp_mesh.FaceMesh(
            static_image_mode=True, 
            max_num_faces=1,
            min_detection_confidence=0.5
        ) as face_mesh:
            results = face_mesh.process(cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB))
        
        # 4. Si no hay landmarks, avisa
        if not results.multi_face_landmarks:
            messagebox.showinfo("Simetría","No se detectó la malla de la cara.")
            return

        # 5. Tomar la malla (un solo rostro)
        face_landmarks = results.multi_face_landmarks[0]
        h, w, _ = face_bgr.shape

        xs = []
        ys = []
        for lm in face_landmarks.landmark:
            xs.append(int(lm.x * w))
            ys.append(int(lm.y * h))
        
        x_left   = min(xs)
        x_right  = max(xs)
        y_top    = min(ys)
        y_bottom = max(ys)

        # 6. Recortar la cara en base al Mesh
        # Asegúrate de no salirte de los límites
        x_left = max(0, x_left)
        x_right = min(w, x_right)
        y_top = max(0, y_top)
        y_bottom = min(h, y_bottom)

        if x_left >= x_right or y_top >= y_bottom:
            messagebox.showinfo("Simetría","La región del mesh es inválida.")
            return

        face_roi = face_bgr[y_top : y_bottom, x_left : x_right]
        hh, ww, _ = face_roi.shape

        # 7. Dividir la cara en dos mitades verticales
        # y hacer el cálculo de MSE (o la métrica que tengas).
        # Asegúrate de que la anchura sea par:
        if ww % 2 != 0:
            ww -= 1
            face_roi = face_roi[:, :ww]  # recorta 1 píxel a la derecha

        mid_x = ww // 2

        left_half = face_roi[:, :mid_x]
        right_half = face_roi[:, mid_x:]

        # 8. Reflejar la mitad derecha
        right_half_flipped = cv2.flip(right_half, 1)

        # 8.1 Guardar las imágenes de la mitad izquierda y derecha reflejada en disco.
        #cv2.imwrite("left_half.jpg", left_half)
        #cv2.imwrite("right_half_flipped.jpg", right_half_flipped)

        # 9. Calcular MSE entre left_half y right_half_flipped
        arr_left = left_half.astype(np.float32)
        arr_right = right_half_flipped.astype(np.float32)
        mse = np.mean((arr_left - arr_right) ** 2)

        # 10. Escalar / interpretar la MSE como porcentaje de simetría
        max_mse = 5000.0
        simmetry_percent = max(0, 100 * (1 - (mse / max_mse)))
        if simmetry_percent > 100:
            simmetry_percent = 100

        # 11. Mostrar resultado
        self.simmetry_label_var.set(f"Simetría: {simmetry_percent:.1f}%")

        # Dibuja líneas, etc. en la ROI para visual:
        col_line = (0,255,255)
        for y_ in range(0, hh, 10):
            cv2.line(face_roi, (mid_x, y_), (mid_x, y_+5), col_line, 1)

        cv2.rectangle(face_roi, (0, 0), (ww-1, hh-1), (0,255,0), 2)
        cv2.putText(face_roi, f"{simmetry_percent:.1f}%", (10,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

        # 12. Mostrar en un popup (por ejemplo)
        show_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(show_rgb)

        win_sym = tk.Toplevel(self)
        win_sym.title("Resultado Simetría Facial (Mesh)")

        sym_label = tk.Label(win_sym)
        sym_label.pack()

        imgtk = ImageTk.PhotoImage(pil_img)
        sym_label.imgtk = imgtk
        sym_label.configure(image=imgtk)

        btn_close = tk.Button(win_sym, text="Cerrar", font=("Helvetica",12),
                            bg="#ff6666", fg="#ffffff",
                            command=win_sym.destroy)
        btn_close.pack(pady=5)

    def show_predictions(self):
        """
            Se analiza con deepfaec edad, género, raza y emocion y se traduce mediante uso de diccionarios los resultados
        """
        if not self.login_image_pil_unprocessed:
            messagebox.showinfo("Predicciones","No hay imagen de login sin procesar.")
            return

        face_bgr=cv2.cvtColor(np.array(self.login_image_pil_unprocessed),cv2.COLOR_RGB2BGR)
        face_rgb=cv2.cvtColor(face_bgr,cv2.COLOR_BGR2RGB)
        try:
            analysis=DeepFace.analyze(img_path=face_rgb,
                                      actions=['age','gender','race','emotion'],
                                      enforce_detection=False)
            age=analysis[0]['age']
            gender=analysis[0]['gender']
            race_en=analysis[0]['dominant_race']
            emo_en=analysis[0]['dominant_emotion']

            gender_info=analysis[0].get("gender",{})
            if isinstance(gender_info,dict):
                man_score=gender_info.get("Man",0.0)
                woman_score=gender_info.get("Woman",0.0)
                if man_score>woman_score:
                    gender_en="Man"
                    gender_conf=man_score
                else:
                    gender_en="Woman"
                    gender_conf=woman_score
            else:
                gender_en="Man" if gender=="Man" else "Woman"
                gender_conf=1.0

            dict_gender={"Man":"Hombre","Woman":"Mujer"}
            gender_es=dict_gender.get(gender_en,"Desconocido")
            conf_str=f"{gender_conf:.3f}"

            dict_race={
                "asian":"Asiático","white":"Blanco",
                "middle eastern":"Medio Oriente","indian":"Indio",
                "latino hispanic":"Latino","black":"Negro"
            }
            race_en_lower=race_en.lower() if isinstance(race_en,str) else""
            race_es=dict_race.get(race_en_lower,race_en)

            dict_emotion={
                "angry":"Enojado","fear":"Miedo","neutral":"Neutral",
                "sad":"Triste","disgust":"Asco","happy":"Feliz","surprise":"Sorpresa"
            }
            emo_en_lower=emo_en.lower() if isinstance(emo_en,str) else""
            emo_es=dict_emotion.get(emo_en_lower,emo_en)

            msg=(f"Edad: {age}\n"
                 f"Género: {gender_es} (conf: {conf_str})\n"
                 f"Raza dominante: {race_es}\n"
                 f"Emoción dominante: {emo_es}")
            self.predictions_label_var.set(msg)
        except Exception as e:
            messagebox.showerror("Error",f"No se pudo analizar: {str(e)}")

    # ---------------------------------------------------------------
    # Stats
    # ---------------------------------------------------------------
    def show_stats_frame(self):
        """Muestra pantalla de similitud (stats)."""
        self.profile_frame.place_forget()
        self.stats_frame.place(x=0,y=0,width=800,height=600)
        self.setup_stats_frame()

    def setup_stats_frame(self):
        for w in self.stats_frame.winfo_children():
            w.destroy()

        lbl_title=tk.Label(self.stats_frame,
                           text="Similitud respecto a cada usuario (tabla)",
                           font=("Helvetica",14,"bold"),bg="#3e3e3e",fg="#ffffff")
        lbl_title.pack(pady=10)

        lbl_header=tk.Label(self.stats_frame,
            text="Nombre             Coseno        Euclidea        Manhattan",
            font=("Courier",12,"bold"),bg="#3e3e3e",fg="#ffffff"
        )
        lbl_header.pack(pady=5)

        if self.login_embedding_vector is None:
            lbl_none=tk.Label(self.stats_frame,
                              text="No hay embedding para calcular similitudes",
                              font=("Helvetica",12),bg="#3e3e3e",fg="#ffffff")
            lbl_none.pack(pady=10)
        else:
            user_dirs=[d for d in os.listdir("usuarios/") if os.path.isdir(os.path.join("usuarios/",d))]
            if not user_dirs:
                lbl_no_users=tk.Label(self.stats_frame,
                                      text="No hay usuarios registrados",
                                      font=("Helvetica",12),bg="#3e3e3e",fg="#ffffff")
                lbl_no_users.pack(pady=10)
            else:
                for user_dir in user_dirs:
                    emb_json_path=os.path.join("usuarios",user_dir,"embeddings.json")
                    if not os.path.exists(emb_json_path):
                        continue
                    with open(emb_json_path,"r") as f:
                        data=json.load(f)
                    user_name_in_json=data["username"]
                    user_embeddings=data["embeddings"]

                    best_cos_dist=float('inf')
                    best_euc_dist=float('inf')
                    best_man_dist=float('inf')

                    for emb_vector in user_embeddings:
                        user_arr=np.array(emb_vector).flatten()
                        if user_arr.shape[0]!=128: 
                            continue

                        dist_cos=df_verification.find_cosine_distance(
                            self.login_embedding_vector,user_arr)
                        dist_euc=euclidean_distance(self.login_embedding_vector,user_arr)
                        dist_man=manhattan_distance(self.login_embedding_vector,user_arr)

                        if dist_cos<best_cos_dist:
                            best_cos_dist=dist_cos
                        if dist_euc<best_euc_dist:
                            best_euc_dist=dist_euc
                        if dist_man<best_man_dist:
                            best_man_dist=dist_man

                    # "Similitud" estilo "porcentaje"
                    cos_sim=(1 - best_cos_dist)*100
                    euc_sim=max(0,100-(best_euc_dist*2.5))
                    man_sim=max(0,100-(best_man_dist*0.5))

                    cos_sim=min(100,max(0,cos_sim))
                    euc_sim=min(100,euc_sim)
                    man_sim=min(100,man_sim)

                    row_text=f"{user_name_in_json:<20}{cos_sim:>6.1f}%{euc_sim:>14.1f}%{man_sim:>14.1f}%"

                    if user_name_in_json==self.current_logged_in_user:
                        lbl_user_stat=tk.Label(
                            self.stats_frame,text=row_text,font=("Courier",12),
                            bg="#aaaaaa",
                            fg="#000000"
                        )
                    else:
                        lbl_user_stat=tk.Label(
                            self.stats_frame,text=row_text,font=("Courier",12),
                            bg="#3e3e3e",fg="#ffffff"
                        )
                    lbl_user_stat.pack(pady=2)

        btn_volver_stats=tk.Button(self.stats_frame,text="Volver",font=("Helvetica",12),
                                   bg="#ff6666",fg="#ffffff",command=self.return_to_profile)
        btn_volver_stats.pack(pady=20)

    # ---------------------------------------------------------------
    # Utilidades
    # ---------------------------------------------------------------
    def show_frame_on_label(self, frame_bgr):
        rgb_frame=cv2.cvtColor(frame_bgr,cv2.COLOR_BGR2RGB)
        imgtk=ImageTk.PhotoImage(Image.fromarray(rgb_frame))
        self.video_label.imgtk=imgtk
        self.video_label.configure(image=imgtk)

    def resize_for_label(self, frame, desired_size):
        target_w, target_h=desired_size
        h,w,_=frame.shape
        aspect_original=w/h
        aspect_target=target_w/target_h

        if aspect_original>aspect_target:
            new_w=int(aspect_target*h)
            x0=(w-new_w)//2
            frame=frame[:, x0:x0+new_w]
            frame=cv2.resize(frame,(target_w,target_h))
        else:
            new_h=int(w/aspect_target)
            y0=(h-new_h)//2
            frame=frame[y0:y0+new_h, :]
            frame=cv2.resize(frame,(target_w,target_h))
        return frame

    def return_to_start(self):
        self.blink_count=0
        self.signup_frame.place_forget()
        self.login_frame.place_forget()
        self.profile_frame.place_forget()
        self.stats_frame.place_forget()
        self.video_label.place_forget()
        self.start_frame.place(x=0,y=0,width=800,height=600)
        self.simmetry_label_var = tk.StringVar(value="")
        self.predictions_label_var = tk.StringVar(value="")


    def return_to_profile(self):
        self.stats_frame.place_forget()
        self.show_profile_frame()

    def calculate_EAR(self, eye_points):
        if len(eye_points)<6:
            return 0
        A=np.linalg.norm(np.array(eye_points[1])-np.array(eye_points[5]))
        B=np.linalg.norm(np.array(eye_points[2])-np.array(eye_points[4]))
        C=np.linalg.norm(np.array(eye_points[0])-np.array(eye_points[3]))
        EAR=(A+B)/(2.0*C) if C!=0 else 0
        return EAR

    def handle_error_sounds(self, eyes_open, mouth_closed):
        """Reproduce audio de error si ojos cerrados/boca abierta y respeta cooldown."""
        now=time.time()
        if self.channel_instructions.get_busy() or self.channel_errors.get_busy():
            return

        if not eyes_open:
            if (now-self.ultimo_sonido_ojos)>=self.audio_cooldown:
                self.pause_background_music()
                self.channel_errors.play(self.audio_ojos)
                dur_ojos=self.audio_ojos.get_length()
                self.after(int(dur_ojos*1000),self.resume_background_music)
                self.ultimo_sonido_ojos=now
            return

        if not mouth_closed:
            if (now-self.ultimo_sonido_boca)>=self.audio_cooldown:
                self.pause_background_music()
                self.channel_errors.play(self.audio_boca)
                dur_boca=self.audio_boca.get_length()
                self.after(int(dur_boca*1000),self.resume_background_music)
                self.ultimo_sonido_boca=now

    def show_flash_effect(self):
        self.flash_label.place(x=0,y=0,width=800,height=600)
        self.after(200,self.flash_label.place_forget)

    def define_ellipse_for_current_offset(self, face_box, frame_size):
        """Posicionamiento y tamaño de óvalos"""
        x_min, y_min, x_max, y_max = face_box
        w, h = frame_size
        face_cx = (x_min + x_max) // 2
        face_cy = (y_min + y_max) // 2

        rx = int((w/3)/1.8)
        ry = int((h/2)/1.8)

        offset_x, offset_y = self.ellipse_offsets[self.current_ellipse_index]

        cx = face_cx + offset_x
        cy = face_cy + offset_y

        # Asegura que "cy" no pase de la mitad de la pantalla:
        cy = min(cy, h // 2)

        # Ajustar para no salirse de la pantalla
        if cx - rx < 0: 
            cx = rx
        if cx + rx > w: 
            cx = w - rx
        if cy - ry < 0: 
            cy = ry
        # aunque definiste cy = min(cy, h//2), podrías dejarle un margen
        if cy + ry > h: 
            cy = h - ry

        self.current_ellipse = (cx, cy, rx, ry)



    """
    Funciones Antiguas utilizadas como pruebas a la hora de trabajar con MediaPipe y el ovalo. 

    Función para ver si el centro de la bounding box de la cara está en el ovalo:
    def is_face_inside_ellipse_by_center(self, face_box, cx, cy, rx, ry):
        x_min, y_min, x_max, y_max = face_box
        face_cx = (x_min + x_max) // 2
        face_cy = (y_min + y_max) // 2
        return self.is_point_in_rotated_ellipse(face_cx, face_cy, cx, cy, rx, ry, 0)


    Comprueba que las 4 esquinas de la bounding box esten dentro:
    def is_face_inside_rotated_ellipse(self, face_box, cx, cy, rx, ry, angle):
        x_min,y_min,x_max,y_max = face_box
        corners=[(x_min,y_min),(x_max,y_min),(x_min,y_max),(x_max,y_max)]
        inside_count=0
        for(xx,yy)in corners:
            if self.is_point_in_rotated_ellipse(xx,yy,cx,cy,rx,ry,angle):
                inside_count+=1
        return (inside_count==4)

    """
    def is_face_inside_rotated_ellipse_mesh(self, mesh_points, cx, cy, rx, ry, angle):
        """
        Se comprueba que se encuentre la face mesh dentro del ovalo
        """
        # mesh_points son 4 tuplas (x,y)
        inside_count = 0
        for (xx, yy) in mesh_points:
            if self.is_point_in_rotated_ellipse(xx, yy, cx, cy, rx, ry, angle):
                inside_count += 1
        # Deben estar dentro los 4 puntos
        return (inside_count == 4) 

    def is_face_inside_rotated_ellipse(self,face_box,cx,cy,rx,ry,angle):
        x_min,y_min,x_max,y_max=face_box
        corners=[(x_min,y_min),(x_max,y_min),(x_min,y_max),(x_max,y_max)]
        inside_count=0
        for(xx,yy)in corners:
            if self.is_point_in_rotated_ellipse(xx,yy,cx,cy,rx,ry,angle):
                inside_count+=1
        return (inside_count==3)    

    def is_point_in_rotated_ellipse(self,x,y,cx,cy,rx,ry,angle):
        xx=x-cx
        yy=y-cy
        import math
        theta=math.radians(-angle)
        cos_t=math.cos(theta)
        sin_t=math.sin(theta)
        x_rot=xx*cos_t - yy*sin_t
        y_rot=xx*sin_t + yy*cos_t
        val=(x_rot**2)/(rx**2)+(y_rot**2)/(ry**2)
        return (val<=1.0)

#--------------------Sección 4: Ejecución de la Aplicación-------------------------------
"""
Finalmente, se instancia la clase principal `App` y se inicia el loop principal 
de la interfaz de usuario (`mainloop`). De esta forma, la aplicación permanece 
corriendo hasta que el usuario la cierre.
"""

if __name__=="__main__":
    app=App()
    app.mainloop()
